In [ ]:
import pandas as pd
import glob
import numpy as np
from tqdm import tqdm
import imblearn
from utils.model_1 import *
from tensorflow import keras
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
testing_combined=pd.read_parquet('combined_sequence_sampled/testing.gzip', engine='auto')
#training_val_combined['labels']=training_val_combined['labels'].astype(int)
testing_combined['measurements']=testing_combined['measurements'].apply(lambda x: np.vstack(x))
X=list(testing_combined['measurements'])
X_padded=tf.keras.utils.pad_sequences(X,dtype=object)

In [ ]:
X_balanced_reshaped=tf.convert_to_tensor(X_padded, dtype=tf.float32)

In [ ]:
model = keras.models.load_model('models/')

In [ ]:
testing_combined=testing_combined[['epoch','measurements','Subject','Session','labels']].rename(columns={'epoch':'time_stamp','labels':'test_pred'})

In [ ]:
y_pred_prob=model.predict(X_balanced_reshaped)
y_pred=np.argmax(y_pred_prob,axis=1)
testing_combined['test_pred']=y_pred

In [ ]:
sub_9_pred=testing_combined[(testing_combined.Subject==10)&(testing_combined.Session==1)].reset_index(drop=True)
sub_9_pred['time_stamp']=sub_9_pred['time_stamp']+0.02
sub_9=pd.read_csv('TestData/subject_010_01__y_time.csv',header=None).rename(columns={0:'time_stamp'})
#sub_9=pd.concat([sub_9,pd.read_csv('TrainingData/subject_012_01__y.csv',header=None)],axis=1).rename(columns={0:'labels'})
#final=pd.concat([sub_9_pred,sub_9],axis=1)[['test_pred','labels']]
#final['truet']=final['test_pred']==final['labels']
#np.sum(final.truet)/(len(final))
#pd.merge(sub_9,sub_9_pred,how='inner',on='time_stamp')
#sub_9_pred['test_pred'].to_csv('predictions/subject_010_01__y.csv',index=False)


In [ ]:
sub_9_pred

In [ ]:
sub_9

In [ ]:
s=pd.read_csv('/Users/moatasimfarooque/Downloads/subject_012_01__y.csv',header=None)

In [ ]:
s_orf=pd.read_csv('predictions/subject_012_01__y.csv')

In [ ]:
s_orf

In [ ]:
p=s[0]==s_orf['test_pred']

In [ ]:
np.sum(p)/len(p)

In [ ]:
s

In [ ]:
s1=pd.read_csv('predictions/subject_012_01__y.csv')

In [ ]:
len(s1)